# 🤖 ODDO BHF Compliance RAG System - Demo

Demonstration interactive du système RAG pour la conformité

## 📦 Setup

In [ ]:
# Imports
from rag_query import ComplianceRAG
from build_vectorstore import VectorStoreBuilder
from data_loader import ComplianceDataLoader
import json

## 🏗️ Build Vector Store (First Time Only)

In [ ]:
# Load data
loader = ComplianceDataLoader()
documents = loader.load_all()

print(f"Loaded {len(documents)} documents")

In [ ]:
# Build vector store (skip if already built)
builder = VectorStoreBuilder()
vectorstore = builder.build(documents, force_rebuild=False)

## 🚀 Initialize RAG System

In [ ]:
# Initialize RAG
rag = ComplianceRAG()
print("✅ RAG system ready!")

## 💬 Example Queries

### Query 1: Règles pour document retail

In [ ]:
result = rag.query("Quelles sont les règles obligatoires pour un document retail français?")

print("📝 Réponse:")
print(result['answer'])
print(f"\n📚 Nombre de sources: {result['num_sources']}")

In [ ]:
# Afficher les sources
print("📚 Sources détaillées:\n")
for i, source in enumerate(result['sources'], 1):
    print(f"{i}. Type: {source['type']}")
    print(f"   ID: {source['id']}")
    print(f"   Fichier: {source['source_file']}")
    if source.get('rule_id'):
        print(f"   Règle: {source['rule_id']} (mandatory: {source.get('mandatory')})")
    print()

### Query 2: Disclaimer spécifique

In [ ]:
result = rag.query("Montre-moi le disclaimer pour un client professionnel allemand")

print("📝 Réponse:")
print(result['answer'])

### Query 3: Validation pays

In [ ]:
result = rag.query("Comment vérifier si un fonds est enregistré dans un pays?")

print("📝 Réponse:")
print(result['answer'])

### Query 4: Performances backtestées

In [ ]:
result = rag.query("Puis-je afficher des performances backtestées?")

print("📝 Réponse:")
print(result['answer'])

## 🔍 Search with Filters

In [ ]:
# Rechercher uniquement les règles obligatoires
docs = rag.search_only(
    "disclaimers",
    k=5,
    filters={"type": "rule", "mandatory": True}
)

print(f"Trouvé {len(docs)} règles obligatoires:\n")
for doc in docs:
    print(f"- {doc.metadata.get('rule_id')}: {doc.page_content[:100]}...")

In [ ]:
# Rechercher disclaimers français pour retail
docs = rag.search_only(
    "disclaimer",
    k=3,
    filters={
        "type": "disclaimer",
        "language": "fr",
        "client_type": "NON_PROFESSIONAL"
    }
)

print(f"Trouvé {len(docs)} disclaimers français retail:\n")
for doc in docs:
    print(f"- {doc.metadata.get('disclaimer_id')}")
    print(f"  {doc.page_content[:150]}...\n")

## 📋 Get Specific Items

In [ ]:
# Récupérer une règle spécifique
rule = rag.get_rule("1.9")

if rule:
    print("📋 Règle 1.9:")
    print(rule['content'])
    print("\nMétadonnées:")
    print(json.dumps(rule['metadata'], indent=2, ensure_ascii=False))

In [ ]:
# Récupérer un disclaimer spécifique
disclaimer = rag.get_disclaimer("FR_PRES_RET_SAS")

if disclaimer:
    print("📄 Disclaimer FR_PRES_RET_SAS:")
    print(disclaimer['content'][:500] + "...")

## ✅ Document Validation

In [ ]:
# Valider un document
validation = rag.validate_document(
    document_type="OBAM_PRESENTATION",
    client_type="retail",
    language="fr",
    country="FR"
)

print("✅ Validation pour présentation retail française:")
print(validation['answer'])
print(f"\n📚 Basé sur {validation['num_sources']} sources")

## 🧪 Custom Queries

In [ ]:
# Votre question personnalisée
question = "Quelles sont les règles spécifiques pour la Belgique?"

result = rag.query(question)
print(f"Question: {question}\n")
print(f"Réponse:\n{result['answer']}")

## 📊 Statistics

In [ ]:
# Statistiques sur les documents
from collections import Counter

# Charger tous les documents
loader = ComplianceDataLoader()
all_docs = loader.load_all()

# Compter par type
types = Counter(doc.metadata['type'] for doc in all_docs)

print("📊 Statistiques des documents:\n")
print(f"Total: {len(all_docs)} documents\n")
for doc_type, count in types.most_common():
    print(f"{doc_type:20s}: {count:3d}")

## 🎉 Ready to Use!

Le système RAG est maintenant opérationnel. Vous pouvez:

- ✅ Poser des questions en langage naturel
- ✅ Rechercher avec des filtres
- ✅ Récupérer des règles/disclaimers spécifiques
- ✅ Valider des documents
- ✅ Analyser les sources

**Pour utilisation en production:**
```python
from rag_query import ComplianceRAG
rag = ComplianceRAG()
result = rag.query("Votre question")
```